## Project Name: Burnt Pizza vs Good Pizza Classification Using ANN

A famous 30 year old pizza brand which has outlets in more than 90 countries started home
delivery services a couple of years ago and the business has grown much faster than expected.
However, outlet vendors are very much disappointed with few customers for their cheating
activities. This is because vendors, shockingly, came to know that few customers after receiving the
delivery are raising tickets for refund in the name of burnt pizzas. Even though customers received
a good pizza but still few customers are trying to cheat vendors. To overcome this issue, Franchise
has come up with an idea to integrate a pizza detection model in their application where customers
can upload images for the burnt pizzas delivered. For example, if I have received a burnt pizza then
I can upload a couple of images of the pizza to the application and it will classify the pizza as burnt
or good in order to process my refund ticket.

**Goal:** You are hired as Deep Learning Engineer by a famous pizza franchise. You are asked to build
a model where it accepts the images of pizza and detects as burnt pizza or good pizza.

**Constraints:** You should be using only ANN and shouldn’t be using CNN or any other rule based
model to generate results.

**Data Description:** Data is in the form of images collected from multiple sources of the internet.
    
**Provided Files:**
**Train set:** Train set is divided into burnt pizza and good pizza categories. While training the
model you can label images of good pizza as 1 and burnt pizza as 0.

**Test:** Test set contains mixed images of both burnt pizzas and good pizzas.
    
**Instructions:**
1. Train set should be used to feed the model.
2. Test set should be used to predict labels for test data.

**Evaluation Criteria:** The evaluation metric for this problem statement is the Accuracy score
where each image label is matched with the actual image label.


**Steps:**
    
1. Importing (or installing) Tenosrflow, Keras and other packages on your system
2. Loading data from disk
3. Creating your training and testing splits
4. Data Preprocessing 
5. Defining your tensorflow ANN model architecture
6. Compiling your tensorflow ANN model
7. Training your model on your training data
8. Evaluating your model on your test data
9. Generate Plots for accuracy and validation loss
10. Saving The train model
11. Making predictions using your trained tensorflow model

**Importing all the packages**

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
%matplotlib inline
import itertools
import warnings
import random
import pickle
import time
import cv2
import os

warnings.filterwarnings("ignore")
SEED=42

**Loading data from disk**

In [ ]:
#initialize the data and labels
print("[INFO] loading images...")
time1=time.time()    # to measure time taken
data = []
labels = []

classes=["Good_pizza","Burnt_pizza"]

#grab the image path and randomly shuffle them
imagePaths = sorted(list(paths.list_images("train")))
random.seed(SEED)
random.shuffle(imagePaths)

#progress bar

with tqdm(total=len(imagePaths)) as pbar:
    
    #loop over the input images
    for imagePath in imagePaths:
        #load the image, resize the image to be 32*32 pixel (ignoring aspect ratio),
        #flatten the 32*32*3 =3072 pixel image into a list, and store image in data list
        image =cv2.imread(imagePath)
        image = cv2.resize(image,(32,32)).flatten()
        data.append(image)
        
        #Extract the class label from the image path and update the label list
        label=imagePath.split(os.path.sep)[-2]
        
        label = 1 if label == "Burnt_pizza" else 0
        labels.append(label)
        
        #update the progressbar
        pbar.update(1)

#scale the raw pixel intensities to the range [0,1]
data=np.array(data,dtype="float")/255.0
labels=np.array(labels)

print("Time Taken: {:.1f} seconds".format(time.time()-time1)) # to measure the time taken
print("done")

In [ ]:
print("Total Images: ",len(data))

In [ ]:
#Sample data for first image
print("Sample image: {}".format(data[0]))
print("no of features/pixels values: {}".format(len(data[0]))) #32*32*3=3072
print("label:{}".format(classes[labels[0]]))


**Creating Training and Testing Splits**

In [ ]:
#partition the data into 80% training and 20 % validation
(trainX,testX,trainY,testY)= train_test_split(data,labels,test_size=0.2,random_state=SEED)

In [ ]:
trainX.shape

In [ ]:
trainY.shape

In [ ]:
testX.shape

In [ ]:
testY.shape

In [ ]:
trainX

In [ ]:
trainY

In [ ]:
type(trainY)

In [ ]:
testY

**Data Preprocessing**

In [ ]:
#converst the Labels from integers/categories to vectors

trainY = to_categorical(trainY,num_classes=2) #fit_transform = find all unique class labels + transform into one hot encoded labels
testY = to_categorical(testY, num_classes=2) # transform = perform the one-hot encoding (unique class labels already found)

# [0,1] Goood_Pizza
# [1,0] Burn_Pizza

In [ ]:
sample_image = (trainX[0]*255).astype("int")

In [ ]:
plt.imshow(sample_image.reshape(32,32,3))

**Define the Architecture for ANN Model**

In [ ]:
#define the 3072-1024-512-1 architecture using keras

model = Sequential()

#input layer 3072 as there are 32*32*3 = 3072 pixels in a flattern input image
#first hidden Layer has 1024 nodes
model.add(Dense(units=1024,input_shape = (3072,),kernel_initializer="uniform",activation='relu'))

#droput for second layer

model.add(Dropout(0.4))

#second hidden layer has 512 nodes
model.add(Dense(units=512,kernel_initializer='uniform',activation='relu'))


#output layer with number of possible class labels
model.add(Dense(units=2,kernel_initializer='uniform',activation='softmax'))


**Compiling Tensorflow ANN model**

In [ ]:
#initialize our initial learning rate and epochs to train for
INIT_LR = 0.01
EPOCHS = 50

#compile the model using SGD as our optimizer and categorical cross_entropy loss
# we have use binary_crossentropy for 2-class classification
print("[INFO] compiling network...")
opt= SGD(lr=INIT_LR) #stochastic Gradient Descent (SGD) optimizer
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])

In [ ]:
model.summary()

**Training Model on Training Data**

In [ ]:
#train the neural network on training data set
#batch_size (32) controls the size of each group of data to pass trough netwaork

time1 = time.time()   # to measure time taken
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=32)
print('Time taken: {:.1f} seconds'.format(time.time() - time1))   # to measure time taken

**Evaluating model on Test Data**

In [ ]:
#evaluate the network
print("[INFO] evaluating network...")
pred_prob=model.predict(testX,batch_size=32)

In [ ]:
pred_prob

In [ ]:
9.95466590e-01, 4.53341054e-03

**Convert testY and y_pred into 1's and 0 for classification Report**

In [ ]:
#note Burn_pizza --> 1 and forest --> 0

In [ ]:
test_y = [np.argmax(i) for i in testY]
pred_y = [np.argmax(i) for i in pred_prob]

In [ ]:
pred_y

In [ ]:
test_y

In [ ]:
def plot_confusion_metrix(y_true, y_pred,classes,
                         normalize=False,
                         title='Confusion Matrix',
                         cmap=plt.cm.Blues):
    """
    Objective
    ----------
    plot confussion matrix, classification report and accuracy score
    
    parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    
    classes : list
        List of labels to index the matrix
        
    title : title for matrix
    cmap : colormap for matrix 
    
    returns 
    ----------
   all accruacy matrix 
    """
    
    
    cm = confusion_matrix(y_true,y_pred)
    
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized Confusion Matrix")
    else:
        print("Confusion Matrix, Without Normalisation")

    
    plt.imshow(cm, interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=35)
    plt.yticks(tick_marks,classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() /2.
    
    for i , j in itertools.product(range(cm.shape[0]), range(cm.shape[0])):
        plt.text(j, i, format(cm[i,j], fmt),
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else 'black')
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.tight_layout()
    
    plt.show()
    
    print("-----------------------------------------------------")
    print('Classification report')
    print(classification_report(y_true,y_pred))
    
    print("-----------------------------------------------------")
    acc= accuracy_score(y_true,y_pred)
    print("Accuracy of the model: ", acc)

In [ ]:
plot_confusion_metrix(test_y,pred_y,classes=["Burn_Pizza","Good_Pizza"])

**Generate Plots for Accuracy and Validation**

In [ ]:
# plot the training and validation loss
N=np.arange(0,EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize = [10,8])
plt.plot(N,H.history["accuracy"],label="Train Accuracy")
plt.plot(N,H.history["val_accuracy"],label="Validation Accuracy")
plt.title("ANN: Training and Validation Accuracy")
plt.xlabel("Epoch #",weight="bold")
plt.legend()
plt.show()

**Saving the Train Model**

In [ ]:
model.summary()

In [ ]:
#save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer..")
model.save("model_ANN.h5")


**Making Predictions using your trained tensorflow model**

In [ ]:
#import the necessary packages
from tensorflow.keras.models import load_model
import pickle
import cv2
import imutils
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#load the model
print("[INFO] loading network and ..")
model = load_model("model_ANN.h5")

In [ ]:
def display_img(img):
    fig = plt.figure(figsize=(12,10))
    plt.grid(b=None)
    ax = fig.add_subplot(111)
    ax.imshow(img)

In [ ]:
# load the input image and resize it to the target spatial dimensions
width = 32
height = 32

# grab the image paths and randomly shuffle them
testImagePaths = sorted(list(paths.list_images('test')))   # test data folder with random images


# progress bar 
with tqdm(total=len(testImagePaths)) as pbar:
    
    for imagePath in testImagePaths:
        image = cv2.imread(imagePath)
        output = image.copy()
        image = cv2.resize(image, (width, height))

        # scale the pixel values to [0, 1]
        image = image.astype("float") / 255.0

        # for a simple fully-connected network, flatten the image
        image = image.flatten()
        image = image.reshape((1, image.shape[0]))


        # make a prediction on the image
        preds = model.predict(image)

        # find the class label index with the largest corresponding probability
        i = preds.argmax(axis=1)[0]
        label = classes[i]
        
        label = "{}: {:.2f}%".format(label, preds[0][i] * 100)

        
        output = imutils.resize(output, width=400)
        cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (0, 255, 0), 2)
        
        # convert img to rgb format and display in noteboo
        img = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
        display_img(img)

#         print("############################")
#         print("image: {}".format(os.path.split(imagePath)[-1]))
#         print("predicted label: {}".format(label))
#         print("Confidence: {}".format(preds[0][i]))
        
        pbar.update(1)

**Deployment**

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
def predict_image(image):
    
    image = cv2.resize(image, (32, 32))

    # scale the pixel values to [0, 1]
    image = image.astype("float") / 255.0

    # for a simple fully-connected network, flatten the image
    image = image.flatten()
    image = image.reshape((1, image.shape[0]))

    # make a prediction on the image
    preds = model.predict(image).flatten()
    result = dict()
    result["Good_Pizza"] = round(float(list(preds)[0]), 3)
    result["Burnt_Pizza"] = round(float(list(preds)[1]), 3)
    print(result)
    
    return result

im = gr.inputs.Image(shape=(32,32))
label = gr.outputs.Label(num_top_classes=2)

gr.Interface(fn=predict_image, inputs=im, outputs=label, capture_session=True, title="Check Pizza Burn or not").launch(share=True)